# Utility

- generate directories to help writing batch job commands

In [1]:
import os

folder = '/home/zhengyang/data/DE/08122021'
sfL = next(os.walk(folder))[1]

for s in sfL:
    print('python extractImages.py ' + os.path.join(folder, s) + ' &')

python extractImages.py /home/zhengyang/data/DE/08122021/2021-08-12_16h32m19s &
python extractImages.py /home/zhengyang/data/DE/08122021/2021-08-12_16h41m05s &
python extractImages.py /home/zhengyang/data/DE/08122021/2021-08-12_16h53m13s &
python extractImages.py /home/zhengyang/data/DE/08122021/channel &
python extractImages.py /home/zhengyang/data/DE/08122021/2021-08-12_16h47m32s &
python extractImages.py /home/zhengyang/data/DE/08122021/2021-08-12_16h44m01s &
python extractImages.py /home/zhengyang/data/DE/08122021/2021-08-12_16h37m10s &


In [6]:
for s in sfL:
    print(os.path.join(folder, s, 'images'))

/home/zhengyang/data/DE/07202021/1st-drop-tracking/2021-07-20_16h47m59s/images
/home/zhengyang/data/DE/07202021/1st-drop-tracking/2021-07-20_16h50m22s/images
/home/zhengyang/data/DE/07202021/1st-drop-tracking/2021-07-20_16h44m58s/images
/home/zhengyang/data/DE/07202021/1st-drop-tracking/2021-07-20_16h35m47s/images
/home/zhengyang/data/DE/07202021/1st-drop-tracking/2021-07-20_16h34m28s/images
/home/zhengyang/data/DE/07202021/1st-drop-tracking/2021-07-20_16h31m06s/images
/home/zhengyang/data/DE/07202021/1st-drop-tracking/2021-07-20_16h28m21s-m4/images
/home/zhengyang/data/DE/07202021/1st-drop-tracking/2021-07-20_16h25m35s-m2/images
/home/zhengyang/data/DE/07202021/1st-drop-tracking/2021-07-20_16h27m01s-m3/images
/home/zhengyang/data/DE/07202021/1st-drop-tracking/2021-07-20_16h29m43s-m5/images
/home/zhengyang/data/DE/07202021/1st-drop-tracking/2021-07-20_16h32m59s/images
/home/zhengyang/data/DE/07202021/1st-drop-tracking/2021-07-20_16h24m06s-mttry/images
/home/zhengyang/data/DE/07202021/1

In [3]:
# check if folder "images" exist
for s in sfL:
    d = os.path.join(folder, s, 'images')
    if os.path.exists(d) == True:
        print(s)

2021-08-04_16h42m12s
2021-08-04_16h43m56s
2021-08-04_16h37m16s
2021-08-04_16h32m08s


## A Progress Bar

In [2]:
print('label [#####################--------------------] 50%')

label [#####################--------------------] 50%


In [4]:
def show_progress(progress, label='', bar_length=60):
    """ Display a progress bar
    Args:
    progress -- float between 0 and 1
    label -- a string displayed before the progress bar
    bar_length -- the length of the progress bar
    
    Returns:
    None
    """
    N_finish = int(progress*bar_length)
    N_unfinish = bar_length - N_finish
    print('{0} [{1}{2}] {3:.1f}%'.format(label, '#'*N_finish, '-'*N_unfinish, progress*100))

In [7]:
show_progress(0.7, label='0812-data')

0812-data [##########################################------------------] 70.0%
